In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

#import numpy as np # linear algebra
#import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [ ]:
from skimage import io
from skimage.transform import  resize
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle


In [ ]:
from os import listdir
import shutil
import os

In [ ]:
train_csv = "../input/hackerearth-deep-learning-identify-the-snake-breed/dataset/train.csv"
test_csv = "../input/hackerearth-deep-learning-identify-the-snake-breed/dataset/test.csv"
train_image_path = "../input/hackerearth-deep-learning-identify-the-snake-breed/dataset/train"
test_image_path = "../input/hackerearth-deep-learning-identify-the-snake-breed/dataset/test"


In [ ]:
train_df_img_list = pd.read_csv(  train_csv, sep = "," )
test_df_img_list =  pd.read_csv(  test_csv, sep = "," )

In [ ]:
train_df_img_list.head()

In [ ]:
fig = plt.figure( figsize = (40,15), dpi = 30)
sns.countplot( data = train_df_img_list, x = "breed", order = train_df_img_list["breed"].value_counts() .index )
plt.xticks( rotation = 90, fontsize = 30 )
plt.yticks( fontsize = 30)
plt.xlabel ("breed", fontsize = 30)
plt.ylabel( "Count", fontsize = 30)

In [ ]:
print ( "Number of class =", len( train_df_img_list["breed"].unique())  )

In [ ]:
''' Creating Data generator '''
def image_read ( path, img_width = 100,img_height = 100 ):
    
    data = io.imread( path)
    return (resize(data/ 255,(img_width, img_height) ) )

In [ ]:
file_list = os.listdir(train_image_path)
plt.imshow ( image_read ( path = train_image_path +"/"+file_list[0],
                        img_width = 100,
                        img_height = 100)
           )

## Data generator, each call will spits out data to save memory

In [ ]:
def data_generator( train_file_name , path, img_height = 100, img_width = 100, batch_size = 300, cat_col = "breed", image_col ="image_id", file_format = ".jpg" ):
    
    df = shuffle( pd.read_csv(train_file_name, sep =",") )
    unique_breed  = list ( df[cat_col].unique() )
    
    while True:
        
        for each_chunk in pd.read_csv(train_file_name, chunksize=batch_size, sep =","):

            data  = np.zeros ( ( each_chunk.shape[0] , img_width, img_height, 3 )   )
            label = np.zeros ( ( each_chunk.shape [0]  , len(unique_breed) )  ) 

            each_chunk= shuffle ( each_chunk )

            for each_row in range( each_chunk.shape[0] ):

                each_image =  path +"/"+ each_chunk.iloc[each_row][0] + file_format

                each_breed =  each_chunk.iloc[each_row][1]

                each_image_data = image_read (path = each_image, img_width = img_width ,img_height = img_height  )

                data[each_row, :, :, 0 ] =  each_image_data[:,:,0]/255

                data[each_row, :, :, 1 ] =  each_image_data[:,:,1]/255

                data[each_row, :, :, 2 ] =  each_image_data[:,:,2]/255

                label[each_row,unique_breed.index(each_breed) ] =  1

            yield ( data, label )

Starting to build model

In [ ]:
''' Required Tensorflow library for preocessing '''
from tensorflow import keras
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout
from tensorflow.keras import Sequential
from tensorflow.keras import preprocessing
import tensorflow as tf

In [ ]:
img_width = 90
img_height = 90
batch_size = 1000
number_of_channel = 3 # RGB
number_of_class = train_df_img_list["breed"].unique()
if ( train_df_img_list.shape[0] % batch_size ) ==0 :
    
    steps_per_epoch = int( train_df_img_list.shape[0]/batch_size)
    
else:
    steps_per_epoch = (train_df_img_list.shape[0] //batch_size ) + 1
    
np.random.random(30)
tf.random.set_seed(30)

train_data_with_label = data_generator(train_file_name = train_csv , 
                            path =train_image_path, 
                            img_height = img_height, 
                            img_width = img_width, 
                            batch_size = batch_size, 
                            cat_col = "breed", 
                            image_col ="image_id", 
                            file_format = ".jpg" )


In [ ]:
cnn_model = Sequential()

cnn_model.add ( Conv2D ( filters = 16, kernel_size = (2,2), strides = (1,1), padding = "valid",activation ="relu", input_shape = ( img_width,img_height, number_of_channel  ) ) )
cnn_model.add (Conv2D  ( filters = 32, kernel_size = (2,2), strides = (2,2), padding = "valid",activation = "elu" )  )
cnn_model.add (Conv2D  ( filters = 32, kernel_size = (2,2), strides = (1,1), padding = "valid",activation = "elu" )  )
cnn_model.add (Dropout(0.2))

cnn_model.add (Conv2D  ( filters = 32, kernel_size = (2,2), strides = (2,2), padding = "valid",activation = "elu" )  )
cnn_model.add (Dropout(0.2))
cnn_model.add (Conv2D  ( filters = 32, kernel_size = (2,2), strides = (2,2), padding = "valid",activation = "elu" )  )
#cnn_model.add (Conv2D  ( filters = 64, kernel_size = (2,2), strides = (1,1), padding = "valid",activation = "elu" )  )
cnn_model.add ( Flatten() )
cnn_model.add ( Dense (units = 128, activation = "relu",use_bias = True  ) )
cnn_model.add (Dropout(0.2))
cnn_model.add ( Dense (units = 64, activation = "relu",use_bias = True  ) )
cnn_model.add (Dropout(0.1))
cnn_model.add ( Dense (units = 35, activation = "softmax",use_bias = False  ) )
cnn_model.summary()

In [ ]:
cnn_model.compile (optimizer= "adam", loss = "categorical_crossentropy" , metrics = ["categorical_accuracy"] )

In [ ]:
cnn_model.fit_generator(  train_data_with_label, steps_per_epoch=steps_per_epoch,epochs = 10, shuffle=  True ) 

In [ ]:
steps_per_epoch